In [5]:
# ===========================
# SIRF PDF FILES KE LINKS 😎
# ===========================

from google.colab import drive
drive.mount('/content/drive')

import os
from google.colab import auth
from googleapiclient.discovery import build

# -------------------------------
# SETTINGS (Sirf yeh change karo)
# -------------------------------
FOLDER_PATH = "/content/drive/MyDrive/urdunovelbank3/urdunovelbank3/Abdullah Hussain Novels"   # <-- Apna folder
OUTPUT_FILE = "/content/drive/MyDrive/novel_links.txt"     # <-- Output file

if not os.path.exists(FOLDER_PATH):
    raise Exception(f"❌ Folder not found: {FOLDER_PATH}")

# -------------------------------
# Drive API setup
# -------------------------------
auth.authenticate_user()
drive_service = build('drive', 'v3')

# -------------------------------
# Folder ka Drive ID nikaalo
# -------------------------------
def get_folder_id_by_path(local_path):
    current_path = local_path.replace("/content/drive/MyDrive", "")
    parts = [p for p in current_path.split("/") if p]

    if not parts:
        return "root"

    folder_id = "root"
    for part in parts:
        query = f"name='{part}' and mimeType='application/vnd.google-apps.folder' and '{folder_id}' in parents and trashed=false"
        result = drive_service.files().list(q=query, fields="files(id)").execute()
        files = result.get('files', [])
        if not files:
            raise Exception(f"Folder not found: {part}")
        folder_id = files[0]['id']
    return folder_id

FOLDER_ID = get_folder_id_by_path(FOLDER_PATH)
print(f"✅ Folder ID: {FOLDER_ID}")

# -------------------------------
# Saare files list karo (recursive)
# -------------------------------
def list_all_files_in_folder(folder_id):
    all_files = []
    page_token = None
    while True:
        results = drive_service.files().list(
            q=f"'{folder_id}' in parents and trashed=false",
            fields="nextPageToken, files(id, name, mimeType)",
            pageSize=1000,
            pageToken=page_token
        ).execute()

        all_files.extend(results.get('files', []))
        page_token = results.get('nextPageToken')
        if not page_token:
            break

    final_files = []
    for item in all_files:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            final_files.extend(list_all_files_in_folder(item['id']))
        else:
            # 🔥 SIRF PDF FILES HI ADD KARO 🔥
            if item['name'].lower().endswith('.pdf'):
                final_files.append(item)
    return final_files

print("🔍 Folder aur subfolders mein sirf PDFs dhoond raha hun...")

all_pdf_files = list_all_files_in_folder(FOLDER_ID)

# -------------------------------
# Sirf PDFs ke links save karo
# -------------------------------
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for file in all_pdf_files:
        file_name = file['name']
        file_id = file['id']
        view_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"

        f.write(f"{file_name}\n")
        f.write(f"{view_link}\n\n")

print(f"🎉 HO GAYA! Sirf {len(all_pdf_files)} PDF files ke links save ho gaye.")
print(f"📄 File: {OUTPUT_FILE}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Folder ID: 1UWnVA0JG4peW8YrIfyb8uJXjEkjldtLu
🔍 Folder aur subfolders mein sirf PDFs dhoond raha hun...
🎉 HO GAYA! Sirf 1 PDF files ke links save ho gaye.
📄 File: /content/drive/MyDrive/novel_links.txt
